# Tabeller för resultat av regressionsanalys

*Av Anders Sundell*

När man gjort en statistisk analys är det viktigt att presentera resultaten på ett överskådligt och pedagogiskt sätt. Ofta handlar det om en kombination av text, tabeller och grafik. Syftet ska vara att presentera så mycket information att läsaren får med sig de viktigaste slutsatserna och har möjlighet att se hur de dragits, men utan att överlasta med för mycket. Särskilt gäller det siffror som man inte tolkar eller kommenterar i texten.

Statas normaloutput efter regressionsanalys är inte lämplig för publicering. Man kan visserligen själv plocka ut de viktigaste uppgifterna och göra en egen tabell i till exempel Word, men det finns enklare sätt, med tilläggskommandon.

Ett sådant är `esttab`. I den här guiden kommer vi gå igenom hur man använder det för att göra en enkel och snygg regressionstabell. Men först måste vi installera esttab, eftersom det inte följer med Stata som standard. Det gör vi genom att skriva följande (och det behöver man bara göra en gång):

In [22]:
ssc install estout, replace

checking estout consistency and verifying not already installed...
installing into /Users/anderssundell/Library/Application Support/Stata/ado/plus/...
installation complete.


Sedan laddar vi in data. I det här exemplet använder vi QoG-institutets QoG Basic data, version 2018. Man kan antingen ladda ned den till sin dator och öppna den där, eller ansluta till den direkt, som jag gör här.

In [2]:
use "https://www.qogdata.pol.gu.se/dataarchive/qog_bas_cs_jan18.dta", clear

(Quality of Government Basic dataset 2018 - Cross-Section)


## Hypotes: Demokrati är bra för livslängd

Analysenheten är här länder. Vi ska göra en enkel analys där vi ska undersöka det möjliga sambandet mellan demokrati och förväntad livslängd. Lever folk längre i demokratier? Och i så fall, håller det sambandet under kontroll för andra variabler, till exempel geografisk placering? Vissa teorier säger att demokratier haft lättare att etablera sig i mer tempererade klimat, långt bort från ekvatorn. Och där är det dessutom mindre tropiska sjukdomar.

Om demokrati är bra för hälsan borde vi finna ett samband mellan demokrati och livslängd, och det sambandet borde hålla även under kontroll för landets geografiska placering.

Variablerna vi ska använda är:
Förväntad livslängd: `wdi_lifexp`
Grad av demokrati: `p_polity2` (-10 till -10)
Avstånd från ekvatorn: `lp_lat_abst`

Nedan visas beskrivande statistik för de tre variablerna.

In [23]:
sum wdi_lifexp p_polity2 lp_lat_abst


    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
  wdi_lifexp |        185    71.25413    8.138066   50.59105   83.58781
   p_polity2 |        165    4.072727    6.158044        -10         10
 lp_lat_abst |        153    .2572459    .1806333          0   .7222222


## Spara resultat av regressionsanalyser med `estimates store`

Vi ska göra tre regressionsanalyser. Först med demokrati som oberoende variabel, sedan med avstånd från ekvatorn, och sedan med både demokrati och avstånd från ekvatorn. Efter varje analys vi gör skriver vi `estimates store m1` där m1 är namnet på en modell (som vi hittar på själv). Jag brukar döpa modellerna till m1, m2, m3 och så vidare. I blocket nedan kör jag de tre analyserna och sparar resultaten efter varje analys.

In [24]:
reg wdi_lifexp p_polity2
estimates store m1

reg wdi_lifexp lp_lat_abst
estimates store m2

reg wdi_lifexp p_polity2 lp_lat_abst
estimates store m3



      Source |       SS           df       MS      Number of obs   =       164
-------------+----------------------------------   F(1, 162)       =     14.93
       Model |  961.395196         1  961.395196   Prob > F        =    0.0002
    Residual |  10432.0079       162  64.3951106   R-squared       =    0.0844
-------------+----------------------------------   Adj R-squared   =    0.0787
       Total |  11393.4031       163  69.8981786   Root MSE        =    8.0247

------------------------------------------------------------------------------
  wdi_lifexp |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
   p_polity2 |   .3942974   .1020468     3.86   0.000      .192784    .5958109
       _cons |   69.20107   .7498878    92.28   0.000     67.72025    70.68188
------------------------------------------------------------------------------



      Source |       SS           df       MS 

## Presentera resultaten med `esttab`

Som synes är resultattabellerna ganska otympliga, och innehåller mycket information som vi vanligtvis inte är intresserade av. Till exempel uttrycker standardfel, t-värde, p-värde och konfidensintervall ungefär samma sak - graden av osäkerhet kring koefficientens skattning. Man behöver inte redovisa alltihopa. Vanligt inom samhällsvetenskap är att man redovisar koefficienten, standardfelet (eller t-värdet) och sätter ut stjärnor som visar signifikansnivån (alltså p-värdet).

`esttab` gör mycket av detta automatiskt. För att göra en minimal tabell av de tre analyserna vi gjort skriver vi bara:

In [26]:
esttab m1 m2 m3


------------------------------------------------------------
                      (1)             (2)             (3)   
               wdi_lifexp      wdi_lifexp      wdi_lifexp   
------------------------------------------------------------
p_polity2           0.394***                        0.155   
                   (3.86)                          (1.49)   

lp_lat_abst                         29.18***        29.33***
                                   (9.17)          (8.23)   

_cons               69.20***        63.62***        62.39***
                  (92.28)         (63.56)         (57.09)   
------------------------------------------------------------
N                     164             147             133   
------------------------------------------------------------
t statistics in parentheses
* p<0.05, ** p<0.01, *** p<0.001


Mycket bättre! Varje kolumn representerar en analys - en "modell." Längst upp anges vad som var beroende variabel i analysen. Siffrorna visar b-koefficienterna för respektive variabel. Inom parenteserna står t-värdet, och längst ner n-talet.

Vi ser då att i den första modellen finns det ett signifikant samband mellan demokrati och förväntad livslängd - varje steg upp på demokratiskalan hänger ihop med en ökning av livslängden på 0.394 år. Men vi ser också i modell 2 att länder som ligger längre bort från ekvatorn har högre förväntad livslängd. Och vi vet (men det syns inte här) att länder längre bort från ekvatorn har högre grader av demokrati. Så när vi kontrollerar för avstånd från ekvatorn i modell 3 ser vi att koefficienten för demokrati mer än halveras, till 0.155, och den är inte heller längre statistiskt signifikant (vilket vi ser eftersom det inte är några stjärnor utsatta, och t-värdet är under 1.96).

Men det finns också annat vi skulle vilja se i tabellen, till exempel R<sup>2</sup>-värdet, eller det justerat R<sup>2</sup>. Och vi kanske hellre vill ha standardfelet inom parenteser, istället för t-värdet. Då kan vi lägga till options på vårt esttab-kommando. Man väljer själv vad man vill lägga till. Använd `help esttab` för att se en fullständig lista.

In [28]:
esttab m1 m2 m3, se r2 ar2


------------------------------------------------------------
                      (1)             (2)             (3)   
               wdi_lifexp      wdi_lifexp      wdi_lifexp   
------------------------------------------------------------
p_polity2           0.394***                        0.155   
                  (0.102)                         (0.104)   

lp_lat_abst                         29.18***        29.33***
                                  (3.183)         (3.562)   

_cons               69.20***        63.62***        62.39***
                  (0.750)         (1.001)         (1.093)   
------------------------------------------------------------
N                     164             147             133   
R-sq                0.084           0.367           0.389   
adj. R-sq           0.079           0.363           0.380   
------------------------------------------------------------
Standard errors in parentheses
* p<0.05, ** p<0.01, *** p<0.001


Den här tabellen är överskådlig och lätt att läsa. Enda problemet är att den inte är så lätt att klistra in i ett Worddokument. Om man gör det måste man sätta typsnittet till courier eller något annat där alla bokstäver är lika stora, annars kommer alla siffror hamna fel.

Men ett bättre sätt är att exportera tabellen till en separat fil som är anpassad för till exempel Word. Då kan man sedan öppna den filen och kopiera tabellen därifrån in i sin rapport.

För att exportera filen lägger vi till `using "filnamn.rtf"` i koden. Då sparas filen i den aktiva mappen. Den anger man genom att skriva `cd "/Users/mindator/statistiskanalys"` till exempel. Jag lägger också till `replace` som option, vilket betyder att om det redan fanns en sådan fil så ska den skrivas över.

In [32]:
esttab m1 m2 m3 using "regressiontabell.rtf", se r2 ar2 replace

(output written to regressiontabell.rtf)


Om man sedan öppnar filen med Word ser det ut på följande sätt:
<img src = "esttabexample.png" width=500>

Sedan kan man naturligtvis göra det ännu mer pedagogiskt genom att byta ut variabelnamnen till förklarande etiketter, till exempel "Demokrati (-10 till 10)" eller motsvarande. Men det här är ändå ett stort lyft jämfört med den råa outputen.

## Avslutning

Att göra tabeller med `esttab` kräver alltså tre steg. Först gör man analysen, sedan sparar man resultaten från den med `estimates store` och sedan presenterar man resultaten med `esttab`.

Tänk på att alltid försöka vara maximalt pedagogisk och tydlig. Den som förlorar mest på att läsaren inte förstår dina resultat är du!